# 4E - four rainfall params

In [1]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
import seaborn as sns
import json
from tensorflow.keras.models import load_model
from scipy.stats import zscore
from sklearn.preprocessing import MinMaxScaler

In [2]:
## Change directory to where your scripts and data file is placed ### 
BASE_PATH = "./private/thesisFinalFiles/"
os.chdir(BASE_PATH)
print(os.path.abspath('')) 

from preprocessing_exp2 import *
from spacetimemodel import spacetimemodel
from preprocessing_exp2 import *
from spacetimemodel import spacetimemodel

/data/private/thesisFinalFiles


In [3]:
data = pd.read_csv('vietnam_dataMatrix_modelling2.csv', header=0)
idx_dic = {}
for col in data.columns:
    idx_dic[col] = data.columns.get_loc(col)
print(idx_dic)
# import sys


{'OID_': 0, 'area': 1, 'landslide': 2, 'su_ID': 3, 'eventDate': 4, 'inventory': 5, 'reliefMax': 6, 'reliefSD': 7, 'planCurMax': 8, 'planCurvSD': 9, 'profCurMax': 10, 'profCurSD': 11, 'slopeMax': 12, 'slopeSD': 13, 'faultDiMax': 14, 'faultDiSD': 15, 'litho': 16, 'landcover': 17, 'F1_max': 18, 'F1_sum': 19, 'F2_max': 20, 'F2_sum': 21, 'F3_max': 22, 'F3_sum': 23, 'F4_max': 24, 'F4_sum': 25, 'F5_max': 26, 'F5_sum': 27, 'F6_max': 28, 'F6_sum': 29, 'F7_max': 30, 'F7_sum': 31, 'F8_max': 32, 'F8_sum': 33, 'F9_max': 34, 'F9_sum': 35, 'F10_max': 36, 'F10_sum': 37, 'F11_max': 38, 'F11_sum': 39, 'F12_max': 40, 'F12_sum': 41, 'F13_max': 42, 'F13_sum': 43, 'F14_max': 44, 'F14_sum': 45, 'F15_max': 46, 'F15_sum': 47, 'F16_max': 48, 'F16_sum': 49, 'F17_max': 50, 'F17_sum': 51, 'F18_max': 52, 'F18_sum': 53, 'F19_max': 54, 'F19_sum': 55, 'F20_max': 56, 'F20_sum': 57, 'F21_max': 58, 'F21_sum': 59, 'F22_max': 60, 'F22_sum': 61, 'F23_max': 62, 'F23_sum': 63, 'F24_max': 64, 'F24_sum': 65, 'F25_max': 66, 'F25

# Preparing rainfall data

## difference

In [4]:
# create a new DataFrame to hold the differences
diffMax_df = pd.DataFrame()
diffSum_df = pd.DataFrame()

df_max = data.iloc[:,np.arange(18,78,2)]
df_sum = data.iloc[:,np.arange(19,78,2)]

# iterate over the columns of the original DataFrame
for i in range(1, len(df_max.columns)):
    if i == 1: 
        diffMax_df[f'diff_{1}'] = 0

    # calculate the difference between this column and the previous column
    diff = df_max.iloc[:, i] - df_max.iloc[:, i-1]
    # add the difference as a new column in the diff_df DataFrame
    diffMax_df[f'diff_{i+1}'] = diff

diffMax_df['diff_1'].fillna(value=0,inplace=True)

for i in range(1, len(df_max.columns)):
    if i == 1: 
        diffSum_df[f'diff_{1}'] = 0

    # calculate the difference between this column and the previous column
    diff = df_sum.iloc[:, i] - df_sum.iloc[:, i-1]
    # add the difference as a new column in the diff_df DataFrame
    diffSum_df[f'diff_{i+1}'] = diff

diffSum_df['diff_1'].fillna(value=0,inplace=True)

# display the resulting DataFrame
diffSum_df.head(10)

diff_1    diff_2    diff_3    diff_4  diff_5  diff_6  diff_7  diff_8  \
0     0.0 -4.912625 -5.044571 -3.623366     0.0     0.0     0.0       0   
1     0.0 -2.692058 -3.221588 -2.186876     0.0     0.0     0.0       0   
2     0.0 -2.061351 -5.082273 -3.681052     0.0     0.0     0.0       0   
3     0.0  0.694908 -3.234181 -1.807860     0.0     0.0     0.0       0   
4     0.0  1.236427 -5.366603 -2.881426     0.0     0.0     0.0       0   
5     0.0  0.185836 -4.513397 -3.030466     0.0     0.0     0.0       0   
6     0.0 -3.521827 -3.073444 -2.824292     0.0     0.0     0.0       0   
7     0.0  0.613479 -1.829478 -0.979057     0.0     0.0     0.0       0   
8     0.0 -4.184481 -3.322984 -3.181882     0.0     0.0     0.0       0   
9     0.0 -2.397909 -2.187063 -1.957479     0.0     0.0     0.0       0   

    diff_9   diff_10  ...   diff_21    diff_22    diff_23   diff_24  \
0  0.00000  8.610378  ...  0.215438 -17.868099   6.540532  3.848720   
1  0.00000  4.465820  ...  0.000000 -11.827802   5.151284  2.553631   
2  0.00000  6.702496  ...  2.936230 -24.051722   9.780282  4.896277   
3  0.77267  0.967182  ...  1.940526 -12.691337   4.967809  2.395039   
4  1.54534  0.552736  ...  3.151703 -19.605534   7.085655  4.749598   
5  0.00000  5.132549  ...  3.295755 -23.070295  10.147362  1.360612   
6  0.00000  4.276779  ...  0.174945 -13.132517   6.057903  2.837765   
7  0.51876  0.518760  ...  1.301463  -6.507314   2.755542  1.786155   
8  0.00000  4.952838  ...  1.075618 -13.967517   5.718315  4.199885   
9  0.00000  2.957720  ...  0.000000  -9.188932   4.311626  1.947706   

     diff_25   diff_26    diff_27    diff_28   diff_29   diff_30  
0 -15.394880  0.979931  10.601730   9.434524 -5.588925  2.961902  
1 -10.214524  0.255634   6.639711   6.098299 -2.396875  0.772670  
2 -15.921331 -1.927152  11.025055   9.234516 -2.575861 -0.362949  
3  -8.927871 -1.367964   6.119515   6.849701 -3.851039  2.069589  
4 -15.098847 -2.233124  10.065898  10.840843 -5.675763  3.378005  
5 -10.574092 -2.114819   8.459274  11.177274 -8.156763  2.718921  
6 -11.133694  0.000000   6.849877   6.648347 -1.687277  0.000000  
7  -5.358467 -0.893078   3.572312   2.769399  0.000000  0.000000  
8 -13.321738  0.000000   6.587186   8.297678 -1.860607  0.000000  
9  -7.790820  0.000000   4.869262   4.632146 -1.187675  0.000000  

[10 rows x 30 columns]

In [5]:
# scaling sum difference 
scaler = MinMaxScaler()
df_sumDiffScaled= scaler.fit_transform(diffSum_df)
df_sumDiffScaled = pd.DataFrame(df_sumDiffScaled, columns=diffSum_df.columns)
df_sumDiffScaled.head(2)

diff_1    diff_2    diff_3    diff_4    diff_5   diff_6   diff_7    diff_8  \
0     0.0  0.279805  0.405322  0.341474  0.258568  0.80072  0.63812  0.636364   
1     0.0  0.363374  0.458661  0.383555  0.258568  0.80072  0.63812  0.636364   

     diff_9   diff_10  ...   diff_21   diff_22   diff_23   diff_24   diff_25  \
0  0.333625  0.722781  ...  0.498392  0.158549  0.596939  0.432004  0.237318   
1  0.333625  0.595618  ...  0.493465  0.283835  0.559144  0.398977  0.359422   

    diff_26   diff_27   diff_28   diff_29   diff_30  
0  0.416769  0.866596  0.685095  0.171108  0.533706  
1  0.379240  0.712965  0.621861  0.250050  0.490652  

[2 rows x 30 columns]

## cumulative sum, cumulative max 

In [6]:
df_cumSumRainfall, df_sumRainfall = cumRainfall(data, 19, 77, 'sum')
df_cumMaxRainfall, df_maxRainfall = cumRainfall(data, 18, 77, 'max')

In [7]:
rainfall_array = rainfallMatrix_4(df_maxRainfall.apply(zscore), df_sumRainfall.apply(zscore), df_cumSumRainfall.apply(zscore), df_sumDiffScaled)

# double checking that the shape matches up what we expected 
rainfall_array.shape

(2759, 30, 4)

# Preparing environmental data

In [8]:
# normalizing continuous static environmental data 
df_staticOnly = data[['litho','landcover',
                    'reliefSD', 'slopeSD', 'cosAspSD', 'sinAspSD', 'faultDiSD',
                     'reliefME', 'plancurME', 'profcurvME', 'slopeME','fautltDiME','sinAspME', 'cosAspME' ]]

catCols = ['landcover','litho']
df_staticOnlyNorm = normCols(df_staticOnly, catCols)
# df_staticOnlyNorm.head(10)

litho          object
landcover      object
reliefSD      float64
slopeSD       float64
cosAspSD      float64
sinAspSD      float64
faultDiSD     float64
reliefME      float64
plancurME     float64
profcurvME    float64
slopeME       float64
fautltDiME    float64
sinAspME      float64
cosAspME      float64
dtype: object 

Index(['reliefSD', 'slopeSD', 'cosAspSD', 'sinAspSD', 'faultDiSD', 'reliefME',
       'plancurME', 'profcurvME', 'slopeME', 'fautltDiME', 'sinAspME',
       'cosAspME'],
      dtype='object')


In [9]:
# one hot encoding the categorical variables in the df 
in_array_env, outDf = catPrep(df_staticOnlyNorm, ['litho','landcover'])

outDf.columns
len(outDf.columns)
env_array = outDf.to_numpy()
env_array.shape

landslide_array = np.array(data['landslide'], dtype=int)


You have one-hot encoded  2  categorical variables. These are: 

litho 

landcover 



# Splitting the data

In [11]:
#split data first into train and test 
idx=np.arange(0,2759)
idx_tr, idx_ts = train_test_split(idx, test_size=0.25,random_state=42)
idx_tr.sort()
idx_ts.sort()

In [12]:
print(len(idx_tr))

2069


# Hyperparameterization

## Hyperband optimization

In [80]:
from optimize_model import * 
import keras_tuner as kt

params= json.load(open('Exp4E_bay_params.json','r'))
print(params)

X_train = [env_array[idx_tr],rainfall_array[idx_tr]]
Y_train = landslide_array[idx_tr]

obj = optimize_model(params['modelparam'])
objfunc = kt.Objective('val_auc', direction='max')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=10, restore_best_weights=True)
tuner2 = kt.Hyperband(obj, objective=objfunc, 
                       max_epochs=150, factor=3, 
                      project_name='msc-tuning-hyperband-4e',
                      overwrite=True, seed=10)
tuner2.search(X_train, Y_train, epochs=200, validation_split=0.2, callbacks=[stop_early]) #tf.keras.callbacks.TensorBoard("/tmp/tb_logs")
best_hps = tuner2.get_best_hyperparameters(num_trials=2)[0]

Trial 254 Complete [00h 13m 36s]
val_auc: 0.6640799045562744

Best val_auc So Far: 0.7634545564651489
Total elapsed time: 05h 33m 21s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [82]:
best_hps.values

{'GRU Layer': 32,
 'Env Layer': 64,
 'Combined Layer': 32,
 'Env depth': 7,
 'Comb depth': 4,
 'learning_rate': 1e-05,
 'GRU activation': 'swish',
 'Env activation': 'tanh',
 'Comb activation': 'relu',
 'gru dropout ratio': 0.1,
 'env dropout ratio': 0.05,
 'comb dropout ratio': 0.1,
 'batch_size': 23,
 'tuner/epochs': 150,
 'tuner/initial_epoch': 50,
 'tuner/bracket': 3,
 'tuner/round': 3,
 'tuner/trial_id': '0207'}

In [109]:
best_model = tuner2.get_best_models(1)[0]

In [111]:
# display model structure
from keras.utils.vis_utils import plot_model
plot_model(best_model, 'best_model.png', show_shapes=True)

# show model summary
best_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 34)]         0           []                               
                                                                                                  
 dense (Dense)                  (None, 64)           2240        ['input_1[0][0]']                
                                                                                                  
 dropout_1 (Dropout)            (None, 64)           0           ['dense[0][0]']                  
                                                                                                  
 dense_1 (Dense)                (None, 64)           4160        ['dropout_1[0][0]']              
                                                                                              

## Bayesian optimization

In [57]:
from optimize_model import * 
import keras_tuner as kt

params= json.load(open('Exp4E_params.json','r'))

X_train = [env_array[idx_tr],rainfall_array[idx_tr]]
Y_train = landslide_array[idx_tr]
objfunc = kt.Objective('val_auc', direction='max')
obj = optimize_model(params['modelparam'])

# note that in older versions, it's obj.build but as of 4/4/23, if build is there, batch size somehow ain't hyperparameterized.
tuner3 = kt.BayesianOptimization(obj, objective=objfunc, 
                      project_name='msc-tuning-exp-4E', max_trials=50, max_retries_per_trial=5,
                      overwrite=True, seed=10)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=10, restore_best_weights=True)
tuner3.search(X_train, Y_train, epochs=200, validation_split=0.2, callbacks=[stop_early])
best_hps3 = tuner3.get_best_hyperparameters(num_trials=2)[0]

Trial 50 Complete [00h 15m 42s]
val_auc: 0.6425853967666626

Best val_auc So Far: 0.7908419966697693
Total elapsed time: 05h 33m 36s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [59]:
best_hps3.values


{'GRU Layer': 16,
 'Env Layer': 64,
 'Combined Layer': 48,
 'Env depth': 6,
 'Comb depth': 3,
 'learning_rate': 1e-05,
 'GRU activation': 'gelu',
 'Env activation': 'tanh',
 'Comb activation': 'softmax',
 'gru dropout ratio': 0.15,
 'env dropout ratio': 0.15,
 'comb dropout ratio': 0.05,
 'batch_size': 17}